#### Requirements and Imports

In [ ]:
!pip install roboflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
!git clone --depth 1 https://github.com/google/automl

Cloning into 'automl'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 181 (delta 16), reused 71 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (181/181), 13.79 MiB | 22.78 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
import os
import sys
import tarfile
from roboflow import Roboflow

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Dataset

In [ ]:
rf = Roboflow(api_key="MHamEhAkUYUWDEpRhEj3")
project = rf.workspace("try2-q1oef").project("solar-module-thermal-anaylsis-it5sj")
version = project.version(1)
dataset = version.download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Solar-Module-Thermal-Anaylsis-1 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 244.47it/s]


In [ ]:
os.chdir('automl/efficientdet')

In [ ]:
!pip install -r requirements.txt
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install pycocotools

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-is9qlz2x
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-is9qlz2x
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-5ut_fjmf
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-5ut_fjmf
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


#### Model Training

In [ ]:
model = 'efficientdet-d4'
batch_size = 4
num_epochs = 25
num_classes = 7

In [ ]:
def download_model_checkpoint(model_name):
  if model_name not in os.listdir():
    checkpoint_url = f'https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco2/{model_name}.tar.gz'
    !wget {checkpoint_url}
    !tar -xzvf {model_name}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), model_name)
  return ckpt_path

In [ ]:
ckpt_path = download_model_checkpoint(model)
print(f"Checkpoint downloaded at {ckpt_path}")

--2024-11-24 00:49:43--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco2/efficientdet-d5.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.207, 142.250.125.207, 209.85.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247255232 (236M) [application/x-tar]
Saving to: ‘efficientdet-d5.tar.gz’

efficientdet-d5.tar 100%[===================>] 235.80M   180MB/s    in 1.3s    

2024-11-24 00:49:45 (180 MB/s) - ‘efficientdet-d5.tar.gz’ saved [247255232/247255232]

efficientdet-d5/
efficientdet-d5/model.meta
efficientdet-d5/d5_coco_test-dev2017.txt
efficientdet-d5/model.index
efficientdet-d5/d5_coco_val.txt
efficientdet-d5/checkpoint
efficientdet-d5/model.data-00000-of-00001
Checkpoint downloaded at /content/automl/efficientdet/efficientdet-d5


In [ ]:
train_tfrecord = '/content/Solar-Module-Thermal-Anaylsis-1/train/solar-pv-thermal.tfrecord'
val_tfrecord = '/content/Solar-Module-Thermal-Anaylsis-1/valid/solar-pv-thermal.tfrecord'

In [ ]:
model_state_path = '/content/drive/MyDrive/PD/EfficientDet/Weights'
os.makedirs(model_state_path, exist_ok=True)

In [ ]:
os.chdir('/content/automl/efficientdet/tf2')
sys.path.append('/content/automl/efficientdet')

In [ ]:
!PYTHONPATH=$PYTHONPATH:/content/automl/efficientdet python train.py --help

2024-11-24 00:49:50.221024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 00:49:50.221114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 00:49:50.222182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 00:49:51.185764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The main training script.
flags:

train.py:
  --batch_size: training batch size
    (default: '64')
    (an integer)
  --[no]debug: Enable debug mode
    (default: 'false')
  --eval_master: GRPC URL of the eval master. Set to an appropriate 

In [ ]:
os.environ['TF_KERAS_USE_LEGACY_BATCH_NORM'] = '1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
print("Starting training...")
!PYTHONPATH=$PYTHONPATH:/content/automl/efficientdet python train.py --model_name=efficientdet-d4 \
  --train_file_pattern=/content/Solar-Module-Thermal-Anaylsis-1/train/solar-pv-thermal.tfrecord \
  --val_file_pattern=/content/Solar-Module-Thermal-Anaylsis-1/valid/solar-pv-thermal.tfrecord \
  --model_dir=/content/drive/MyDrive/PD/EfficientDet/Weights \
  --batch_size=4 \
  --num_epochs=25 \
  --num_examples_per_epoch=1000 \
  --eval_samples=100 \
  --hparams="learning_rate=0.08,momentum=0.9,moving_average_decay=0,optimizer=sgd"

Starting training...
2024-11-24 01:29:41.764595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 01:29:41.764642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 01:29:41.766820: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 01:29:42.845714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1124 01:29:45.129416 132818411921408 efficientnet_builder.py:215] global_params= GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.4, data_format='channels_last', num_classes=1000, width_co

#### Model Evaluation

In [ ]:
os.chdir('/content/automl/efficientdet')

!PYTHONPATH=$PYTHONPATH:/content/automl/efficientdet python main.py --mode=eval \
  --model_name=efficientdet-d4 \
  --model_dir=/content/drive/MyDrive/PD/EfficientDet/Weights \
  --val_file_pattern=/content/Solar-Module-Thermal-Anaylsis-1/valid/solar-pv-thermal.tfrecord \
  --val_json_file=/path/to/validation/annotations.json \
  --eval_samples=1000 \
  --hparams="num_classes=7"

2024-11-24 00:51:57.497925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 00:51:57.498060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 00:51:57.648773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 00:52:00.447622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1124 00:52:05.481978 135219705798656 main.py:228] {'name': 'efficientdet-d5', 'act_type': 'swish', 'image_size': (1280, 1280), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 

In [ ]:
!PYTHONPATH=$PYTHONPATH:/content/automl/efficientdet python model_inspect.py \
  --runmode=saved_model \
  --model_name=efficientdet-d4 \
  --ckpt_path=/content/drive/MyDrive/PD/EfficientDet/Weights \
  --saved_model_dir=/tmp/saved_model \
  --hparams="num_classes=7"

2024-11-24 01:12:43.631061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 01:12:43.631156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 01:12:43.634895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 01:12:44.799295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.NEAREST_NEIGHBOR...)` instead.
W1124 01:12:55.150419 136089364221952 deprecation.py:50] From /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:12

In [ ]:
gpu_memory_gb = 5.57
total_training_time = 1.25*3600

In [ ]:
cost = ((gpu_memory_gb/15)*69.55) * (total_training_time/3600)
print(cost)

32.28279166666667
